In [147]:
import pandas as pd
import geopandas
import datetime

In [148]:
df = pd.read_csv("daily_vi_soum_2020.csv")
df.head()

,system:index,asid,evi_count,evi_int_mean_10_90,evi_max,evi_mean,evi_median,evi_min,evi_stdDev,msavi_count,...,nirv_min,nirv_stdDev,savi_count,savi_int_mean_10_90,savi_max,savi_mean,savi_median,savi_min,savi_stdDev,.geo
0,1_LC08_122027_20200202_000000000000000000cc,2128.0,1896364,0.818394,3.141120e+14,7.948449e+08,4.466502,-3.134153e+14,6.512603e+11,1896207,...,-18.397412,0.043519,1896364,-0.056771,0.974561,-0.056747,-0.058736,-0.871729,0.027742,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1_LC08_122027_20200218_000000000000000000cc,2128.0,2107658,1.032801,2.759510e+14,1.022942e+08,5.223120,-2.689845e+14,5.388028e+11,2107658,...,-0.589886,0.015685,2107658,-0.056753,0.495613,-0.054675,-0.056719,-0.397588,0.023644,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,1_LC08_122027_20200305_000000000000000000cc,2128.0,2409110,1.355963,2.580703e+14,-2.884423e+08,4.508037,-2.436729e+14,4.990205e+11,2409110,...,-1.464130,0.013599,2409110,-0.050857,0.470866,-0.048394,-0.052842,-0.477636,0.021580,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,1_LC08_122027_20200321_000000000000000000cc,2128.0,2419822,-1.427449,3.083840e+14,1.013436e+08,-9.985732,-2.959991e+14,1.028633e+12,2419822,...,-0.127363,0.016451,2419822,-0.054733,0.484807,-0.050540,-0.056640,-0.467703,0.029375,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,1_LC08_122027_20200406_000000000000000000cc,2128.0,2555098,0.119564,3.548008e+00,1.106312e-01,0.113708,-1.435888e+00,4.292431e-02,2555098,...,-8.157415,0.017705,2555098,0.127311,0.489273,0.119362,0.134781,-0.233309,0.046865,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [149]:
def parseDate(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function parses date from first column

    Args:
        df (pd.DataFrame):

    Returns:
        pd.DataFrame:
    """
    df["date"] = df["system:index"].str.split("_").str[-2]
    df["date"] = pd.to_datetime(df['date'])

    return df

In [150]:
df = parseDate(df)
df.head()
len(df)

30998

In [151]:
df.head()

,system:index,asid,evi_count,evi_int_mean_10_90,evi_max,evi_mean,evi_median,evi_min,evi_stdDev,msavi_count,...,nirv_stdDev,savi_count,savi_int_mean_10_90,savi_max,savi_mean,savi_median,savi_min,savi_stdDev,.geo,date
0,1_LC08_122027_20200202_000000000000000000cc,2128.0,1896364,0.818394,3.141120e+14,7.948449e+08,4.466502,-3.134153e+14,6.512603e+11,1896207,...,0.043519,1896364,-0.056771,0.974561,-0.056747,-0.058736,-0.871729,0.027742,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-02-02
1,1_LC08_122027_20200218_000000000000000000cc,2128.0,2107658,1.032801,2.759510e+14,1.022942e+08,5.223120,-2.689845e+14,5.388028e+11,2107658,...,0.015685,2107658,-0.056753,0.495613,-0.054675,-0.056719,-0.397588,0.023644,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-02-18
2,1_LC08_122027_20200305_000000000000000000cc,2128.0,2409110,1.355963,2.580703e+14,-2.884423e+08,4.508037,-2.436729e+14,4.990205e+11,2409110,...,0.013599,2409110,-0.050857,0.470866,-0.048394,-0.052842,-0.477636,0.021580,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-03-05
3,1_LC08_122027_20200321_000000000000000000cc,2128.0,2419822,-1.427449,3.083840e+14,1.013436e+08,-9.985732,-2.959991e+14,1.028633e+12,2419822,...,0.016451,2419822,-0.054733,0.484807,-0.050540,-0.056640,-0.467703,0.029375,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-03-21
4,1_LC08_122027_20200406_000000000000000000cc,2128.0,2555098,0.119564,3.548008e+00,1.106312e-01,0.113708,-1.435888e+00,4.292431e-02,2555098,...,0.017705,2555098,0.127311,0.489273,0.119362,0.134781,-0.233309,0.046865,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-04-06


In [152]:
def getMongoliaShpFile(filepath: str="soum_aimag.shp") -> pd.DataFrame:
  gdf = geopandas.read_file("soum_aimag.shp")
  df = pd.DataFrame(gdf)
  return df


In [153]:
def joinShpData(df1, df2, partition: str="asid"):
  df2["asid"] = df2["asid"].astype("int")
  df1["asid"] = df1["asid"].astype("int")
  df = pd.merge(df1, df2, on=partition, how="outer")
  return df

In [155]:
df1 = getMongoliaShpFile()
df["asid"] = df["asid"].astype("int")
df1["asid"] = df1["asid"].astype("int")
df = joinShpData(df, df1)
df.head()

,system:index,asid,evi_count,evi_int_mean_10_90,evi_max,evi_mean,evi_median,evi_min,evi_stdDev,msavi_count,...,savi_min,savi_stdDev,.geo,date,au2_code,aid,aimag_eng,sid,soum_eng,geometry
0,1_LC08_122027_20200202_000000000000000000cc,2128,1896364.0,0.818394,3.141120e+14,7.948449e+08,4.466502,-3.134153e+14,6.512603e+11,1896207.0,...,-0.871729,0.027742,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-02-02,02128,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."
1,1_LC08_122027_20200218_000000000000000000cc,2128,2107658.0,1.032801,2.759510e+14,1.022942e+08,5.223120,-2.689845e+14,5.388028e+11,2107658.0,...,-0.397588,0.023644,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-02-18,02128,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."
2,1_LC08_122027_20200305_000000000000000000cc,2128,2409110.0,1.355963,2.580703e+14,-2.884423e+08,4.508037,-2.436729e+14,4.990205e+11,2409110.0,...,-0.477636,0.021580,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-03-05,02128,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."
3,1_LC08_122027_20200321_000000000000000000cc,2128,2419822.0,-1.427449,3.083840e+14,1.013436e+08,-9.985732,-2.959991e+14,1.028633e+12,2419822.0,...,-0.467703,0.029375,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-03-21,02128,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."
4,1_LC08_122027_20200406_000000000000000000cc,2128,2555098.0,0.119564,3.548008e+00,1.106312e-01,0.113708,-1.435888e+00,4.292431e-02,2555098.0,...,-0.233309,0.046865,"{""type"":""MultiPoint"",""coordinates"":[]}",2020-04-06,02128,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."


In [156]:
keep_columns = ["date", "asid", "ndvi_count", "ndvi_int_mean_10_90", "ndvi_max", "ndvi_mean", \
                "ndvi_median", "ndvi_min", "ndvi_stdDev", \
                "aid", "aimag_eng", "sid", "soum_eng", "geometry"]

In [157]:
ndvi_df = df[keep_columns]

In [158]:
ndvi_df.head()

,date,asid,ndvi_count,ndvi_int_mean_10_90,ndvi_max,ndvi_mean,ndvi_median,ndvi_min,ndvi_stdDev,aid,aimag_eng,sid,soum_eng,geometry
0,2020-02-02,2128,1894021.0,-0.047326,0.999473,-0.048933,-0.050792,-0.998314,0.031170,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."
1,2020-02-18,2128,2107517.0,-0.047549,0.999891,-0.046223,-0.050743,-0.918397,0.024627,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."
2,2020-03-05,2128,2409043.0,-0.042600,0.997688,-0.040631,-0.043350,-0.572512,0.022403,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."
3,2020-03-21,2128,2419802.0,-0.046405,0.985806,-0.041982,-0.043407,-0.559611,0.035878,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."
4,2020-04-06,2128,2555070.0,0.207596,0.964926,0.194381,0.222771,-0.435935,0.080214,21.0,Dornod,28.0,Khalkhgol,"POLYGON ((119.00437 46.74707, 119.00192 46.745..."


In [ ]:
ndvi_df.to_csv('ndvi.csv') 